In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64


# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

image_filename = "Grazioso Salvare Logo.png"
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#### FIX ME : Attempted ##### Attempted -GG
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animalShelter import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name. NOTE: You will
# likely need more variables for your constructor to handle the hostname and port of the MongoDB
# server, and the database and collection names

# Attempted - GG.

username = "aacuser"
password = "SNHU1234"
host = 'nv-desktop-services.apporto.com'
port = 31060
database = 'AAC'
collection = 'animals'


shelter = AnimalShelter(username, password, host, port, database, collection)


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Project - Gerardo Gonzalez'))),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    html.Hr(),
    html.Div(className='buttonRow', style={'display':'flex'},
            children=[
                html.Button(id='submit-button-one',   n_clicks=0, children='Water'),
                html.Button(id='submit-button-two',   n_clicks=0, children='Wilderness'),
                html.Button(id='submit-button-three', n_clicks=0, children='Disaster'),
                html.Button(id='submit-button-four',  n_clicks=0, children='Reset')
                     ]),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable = False,
        filter_action = "native", 
        sort_action = "native", 
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = "single", 
        row_deletable = False,
        selected_columns = [],
        selected_rows = [0],
        page_action = "native",
        page_current = 0,
        page_size = 10,
        

    ),
     html.Br(),
     html.Hr(),
     html.Div(
            id='graph-id',
            className='col s12 m6',
            ),
     html.Div(
            id='map-id',
            className='col s12 m6',
            ),
     html.Div(id="query-out", style={'whiteSpace': 'pre-line'}), html.H4("Module 7 - Gerardo Gonzalez")
])

#############################################
# Interaction Between Components / Controller
#############################################
@app.callback(Output('datatable-id', "data"),
             [Input('submit-button-one', 'n_clicks'),
              Input('submit-button-two', 'n_clicks'),
              Input('submit-button-three', 'n_clicks'),
              Input('submit-button-four', 'n_clicks')
             ])
def on_click(button1, button2, button3, button4):
    df = pd.DataFrame.from_records(shelter.read({}))
    count = 1
    
    if (int(button1) > 0):
        df = pd.DataFrame.from_records(shelter.read({
            "animal_type" : "Dog",
            "breed": {"$in":["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},                                         
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        }))
    if (int(button2) > 0):
        df = pd.DataFrame.from_records(shelter.read({
            "animal_type" : "Dog",
            "breed": {"$in":["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }))
    if(int(button3) > 0):
        df = pd.DataFrame.from_records(shelter.read({
            "animal_type" : "Dog",
            "breed": {"$in":["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
            }))
    if(int(button4) > 0):
        df = pd.DataFrame.from_records(shelter.read({}))
        
        
    df.drop(columns=['_id'],inplace=True)
    return df.to_dict('records')


#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    if viewData is None:
        return
    dff = pd.DataFrame.from_dict(viewData)
    pie_chart = dcc.Graph(
        figure = px.pie(dff, names='breed', title='Breed Distrubution'),
        style = {'width': '1000px','height': '400px'}
    )
    
    return [pie_chart]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    #pass
#FIXME Add in the code for your geolocation chart
    dff=pd.DataFrame.from_dict(viewData)
    print("Number of Rows in DataFrame:", dff.shape[0])
    if index is None or not index or len(dff) ==0:
        row=0
        print("Number of Rows:", len(dff))
    else:
        row=index[0]
        
    if row >= len(dff):
    # Handle the case where row index is out of bounds
        return []
    return [
        dl.Map(style={'width': '1000px', 'height' : '500px'},
              center=[30.75, -97.48], zoom=9, children=[
              dl.TileLayer(id="base-layer-id"),
              dl.Marker(position=[dff.iloc[row,13], dff.iloc[row,14]],          
              children=[dl.Tooltip(dff.iloc[row,4]),dl.Popup([html.H1("Animal Name"),html.P(dff.iloc[row, 9])
            ])
        ])
    ])
]
            


app.run_server(debug=True)

Dash app running on http://127.0.0.1:12326/
Number of Rows in DataFrame: 0
Number of Rows: 0
Number of Rows in DataFrame: 30008
Number of Rows in DataFrame: 12
